#  Taylor diagram

+ author : Stephanie Leroux, Ocean Next, MEOM
+ mail : stephanie.leroux@ocean-next.fr
+ date : Jun 2018
+ purpose :  Demo for plotting a Taylor diagram using a code from http://www-pcmdi.llnl.gov/about/staff/Taylor/CV/Taylor_diagram_primer.htm
+ Note: Many improvments needed!

### obs:
```
# xarray containing the obs ts
tsobs = dat.sel(e=1,x=80)
<xarray.DataArray 'ssh' (time_counter: 4088)>
array([-0.032042, -0.031983, -0.03191 , ..., -0.008232, -0.011431, -0.008957],
      dtype=float32)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 1960-01-03T12:00:00 ...
    SXloc         <U4 'zSX1'
```

### model:
```
# xarray containing 48 ts from ensemble members 
tsee   = dat.sel(e=slice(2,50),x=80)
<xarray.DataArray 'ssh' (e: 48, time_counter: 4088)>
array([[-0.010992, -0.010965, -0.010909, ...,  0.032176,  0.037695,  0.050215],
       [ 0.001269,  0.001241,  0.001162, ...,  0.139489,  0.097042,  0.09971 ],
       [ 0.030129,  0.03008 ,  0.03004 , ..., -0.0306  , -0.029764, -0.028869],
       ...,
       [-0.006796, -0.006767, -0.006697, ...,  0.013105,  0.016415,  0.017743],
       [ 0.010819,  0.010847,  0.010906, ..., -0.191347, -0.173172, -0.156769],
       [ 0.00072 ,  0.000715,  0.000607, ..., -0.001673,  0.003151,  0.019461]],
      dtype=float32)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 1960-01-03T12:00:00 ...
    SXloc         <U4 'zSX1'
Dimensions without coordinates: e
```

### Function from [this website](http://www-pcmdi.llnl.gov/about/staff/Taylor/CV/Taylor_diagram_primer.htm)

In [ ]:
#!/usr/bin/env python
# Copyright: This document has been placed in the public domain.

"""
Taylor diagram (Taylor, 2001) test implementation.
http://www-pcmdi.llnl.gov/about/staff/Taylor/CV/Taylor_diagram_primer.htm
"""

__version__ = "Time-stamp: <2012-02-17 20:59:35 ycopin>"
__author__ = "Yannick Copin <yannick.copin@laposte.net>"

import numpy as NP
import matplotlib.pyplot as PLT


class TaylorDiagram(object):
    """Taylor diagram: plot model standard deviation and correlation
    to reference (data) sample in a single-quadrant polar plot, with
    r=stddev and theta=arccos(correlation).
    """

    def __init__(self, refstd,  fig=None, rect=111, label='_', refsty='no',stdmax=1.5):
        """Set up Taylor diagram axes, i.e. single quadrant polar
        plot, using mpl_toolkits.axisartist.floating_axes. refstd is
        the reference standard deviation to be compared to.
        """

        from matplotlib.projections import PolarAxes
        import mpl_toolkits.axisartist.floating_axes as FA
        import mpl_toolkits.axisartist.grid_finder as GF

        self.refstd = refstd            # Reference standard deviation

        tr = PolarAxes.PolarTransform()

        # Correlation labels
        rlocs = NP.concatenate((NP.arange(10)/10.,[0.95,0.99]))
        tlocs = NP.arccos(rlocs)        # Conversion to polar angles
        gl1 = GF.FixedLocator(tlocs)    # Positions
        tf1 = GF.DictFormatter(dict(zip(tlocs, map(str,rlocs))))

        # Standard deviation axis extent
        self.smin = 0
        self.smax = stdmax*self.refstd

        ghelper = FA.GridHelperCurveLinear(tr,
                                           extremes=(0,NP.pi/2, # 1st quadrant
                                                     self.smin,self.smax),
                                           grid_locator1=gl1,
                                           tick_formatter1=tf1,
                                           )

        if fig is None:
            fig = PLT.figure()

        ax = FA.FloatingSubplot(fig, rect, grid_helper=ghelper)
        fig.add_subplot(ax)

        # Adjust axes
        ax.axis["top"].set_axis_direction("bottom")  # "Angle axis"
        ax.axis["top"].toggle(ticklabels=True, label=True)
        ax.axis["top"].major_ticklabels.set_axis_direction("top")
        ax.axis["top"].label.set_axis_direction("top")
        ax.axis["top"].label.set_text("Correlation")

        ax.axis["left"].set_axis_direction("bottom") # "X axis"
        ax.axis["left"].label.set_text("Standard deviation")

        ax.axis["right"].set_axis_direction("top")   # "Y axis"
        ax.axis["right"].toggle(ticklabels=True)
        ax.axis["right"].major_ticklabels.set_axis_direction("left")

        ax.axis["bottom"].set_visible(False)         # Useless
        
        # Contours along standard deviations
        ax.grid(False)

        self._ax = ax                   # Graphical axes
        self.ax  = ax.get_aux_axes(tr)   # Polar coordinates

        # Add reference point and stddev contour
        print("Reference std:", self.refstd)
        if refsty in ('OsqO','YtriE','GplM'):
            if refsty in ('OsqO'):
                colorstyle="#fe642e"
                l, = self.ax.plot([0], self.refstd, 's',color=colorstyle,markeredgecolor=colorstyle,ls='',ms=10, label=label)
                
                
            if refsty in ('YtriE'):
                colorstyle="#facc2e"
                l, = self.ax.plot([0], self.refstd, '^',color=colorstyle,markeredgecolor='none',ls='', ms=15, label=label)

            if refsty in ('GplM'):
                colorstyle="#424242"
                l, = self.ax.plot([0], self.refstd, '+',color="k",
                          ls='', ms=10, label=label,mew=3.5)
        else:
            colorstyle="r"
            l, = self.ax.plot([0], self.refstd, 's',color=colorstyle,markeredgecolor="k",ls='', ms=8, label=label)

        t = NP.linspace(0, NP.pi/2)
        r = NP.zeros_like(t) + self.refstd
        self.ax.plot(t,r, color='k', linewidth=1.5,linestyle="-",label='_')
        #self.ax.plot(t,r, color="gray", label='_')

        # Collect sample points for latter use (e.g. legend)
        
        self.samplePoints = [l]





    def add_sample(self, stddev, corrcoef, *args, **kwargs):
        """Add sample (stddev,corrcoeff) to the Taylor diagram. args
        and kwargs are directly propagated to the Figure.plot
        command."""

        l, = self.ax.plot(NP.arccos(corrcoef), stddev,
                          *args, **kwargs) # (theta,radius)
        self.samplePoints.append(l)

        return l

    def add_contours(self, levels=5, **kwargs):
        """Add constant centered RMS difference contours."""

        rs,ts = NP.meshgrid(NP.linspace(self.smin,self.smax),
                            NP.linspace(0,NP.pi/2))
        # Compute centered RMS difference
        rms = NP.sqrt(self.refstd**2 + rs**2 - 2*self.refstd*rs*NP.cos(ts))
        
        contours = self.ax.contour(ts, rs, rms, levels, **kwargs)

        return contours


    
    
    

if __name__=='__main__':

    # Reference dataset
    x = NP.linspace(0,4*NP.pi/2,100)
    data = NP.sin(x)
    refstd = data.std(ddof=1)           # Reference standard deviation

    # Models
    m1 = data + 0.2*NP.random.randn(len(x))    # Model 1
    m2 = 0.8*data + .1*NP.random.randn(len(x)) # Model 2
    m3 = NP.sin(x-NP.pi/10)                    # Model 3

    # Compute stddev and correlation coefficient of models
    samples = NP.array([ [m.std(ddof=1), NP.corrcoef(data, m)[0,1]]
                         for m in (m1,m2,m3)])

    fig = PLT.figure(figsize=(10,4))
    
    ax1 = fig.add_subplot(1,2,1, xlabel='X', ylabel='Y')
    # Taylor diagram
    dia = TaylorDiagram(refstd, fig=fig, rect=122, label="E-mean")

    colors = PLT.matplotlib.cm.jet(NP.linspace(0,1,len(samples)))

    ax1.plot(x,data,'ko', label='Data')
    for i,m in enumerate([m1,m2,m3]):
        ax1.plot(x,m, c=colors[i], label='Model %d' % (i+1))
    ax1.legend(numpoints=1, prop=dict(size='small'), loc='best')

    # Add samples to Taylor diagram
    for i,(stddev,corrcoef) in enumerate(samples):
        dia.add_sample(stddev, corrcoef, marker='s', ls='', c=colors[i],
                       label="Model %d" % (i+1))

    # Add RMS contours, and label them
    contours = dia.add_contours(colors='0.85')
    PLT.clabel(contours, inline=1, fontsize=10)

    # Add a figure legend
    fig.legend(dia.samplePoints,
               [ p.get_label() for p in dia.samplePoints ],loc=3,
               numpoints=1, prop=dict(size='small'))

    
    PLT.show()

## Example of how i use it :

In [ ]:
pltname='demo'
    #---- prepare plot
refsty = 'OsqO' #plot style

#### THE REFERENCE TIMESERIES  IS OBS
stdrefs = 1. 
refts   = tsobs
norm    =  refts.std(dim='time_counter',ddof=1)

#### normalization by the std of the ref timeseries:

# ref ts
refts   =  tsobs/norm
refts = refts.values

# N=50 x ts (occiput)
maints = tsee/norm
maints = maints.values



### prepare dict of points to plot: Tstd,correlation,'name'

# occiput
samples = dict(Sample1=[[maints[im,:].std(ddof=1), np.corrcoef(refts,maints[im,:])[0,1],im]
          for im in range(0,48)])

# std max on the plot (circle size)
stdmax = 2.0 #max(df_dtdsc_1D_N50.std(ddof=1,axis=0)/norm)

#rects = dict(Sample1=111) 
namedic = "Sample1" 


#### start plotting
plt.close('all')
fig = plt.figure(figsize=(8,5),facecolor='white')

# create normalized diagram with x-axis going to stdmax
dia = TaylorDiagram(stdrefs, fig=fig, rect=111,label='REF = OBS',refsty=refsty,stdmax=stdmax)
contours = dia.add_contours(levels=10, colors='#D8D8D8',linewidth=0.5) 
dia.ax.clabel(contours, inline=1, fontsize=9, fmt='%.2f')

# add a fake point (0,0) with label for the ensemble members
dia.add_sample(0,0,marker='x',ms=5,ls='',label="Individual members",mew=1.2,mfc="#086A87",mec='#424242')       



# add legend
plt.rcParams['legend.frameon'] = False  

fig.legend(dia.samplePoints,
                   [ p.get_label() for p in dia.samplePoints ],
                    numpoints=1, prop=dict(size='small'), loc=3,bbox_to_anchor=(0.55, 0.82),
                    ncol=1,  borderaxespad=0.,frameon=False, mode="expand")
plt.rcParams['legend.frameon'] = False           
  


# add individual members
for i,(stddev,corrcoef,name) in enumerate(samples[namedic]):
      dia.add_sample(stddev, corrcoef,marker='x', ms=5, ls='',mew=1.2,mfc="#585858", mec='#585858',zorder=30)

fig.tight_layout()
plt.savefig(diro+pltname+"."+suffix+"."+pltty)
plt.show()

